## Modelo de SeleAI, utilizar predição para classificar candidatos
- **Objetivo:** Criar padronização de formulário para abertura de vagas e aplicações. No formulário da vaga terá todas as informações necessárias da vaga, essas informações serão amarzenadas em um json com um ID sequencial unico da vaga e serão utilizadas no portal do candidato. Ao preencher as informações o consultor criará um formulário automático para que o candidato possa preencher que será ativado quando o candidato preencher para qual vaga ele está se candidatando (id + titulo_vaga). Ao se candidatar para a vaga, todas as informações serão armazenadas em outro json, com o nome aplicantes ele terá o código do candidato linkado a vaga. 

O resultado desses dois formulários aparecerá em um dashboard de match, no qual classificará os candidatos com base nas features abaixo, treinadas por um modelo de machine ou deep learning, com similaridade de palavras e intervalos numericos. Formularios no streamlit e json no github. Dashboard em excel, consultando de uma tabela de classificação de candidatos por vaga.

### **Features de avaliação**

- **Fator Salarial:**
Comparar a pretenção salarial do candidato com o target orçamentário da vaga. Se a pretenção estiver dentro do intervalo, 1.0, se for 20% menor que limite inferior
ou 10% maior que o limite superior, então 0,5, se não estiver em nenhuma das condições anteriores 0.
- **Fator Engajamento:**
Estilo de trabalho, se a preferencia do candidato for igual a da vaga, então 1 (hibrido, remoto ou presencial). Se o trabalho houver compatibilidade entre os benefícios (similaridade de nomes) ou o da vaga for superior, então +1, caso o do emprego atual do funcionário seja maior, então 0. Se preferir o tipo de contrato igual a vaga +1 ponto, se não +0 (clt ou PJ)
- **Fator cultural:** 
Compatibilidade de perfil com as competências pessoais exigidas na vaga. Exemplo se a vaga pede alguém com liderança e o candidato marcar liderança, então isso contribui para a similaridade.
- **Fator técnico:** 
Compatibilidade de perfil ténico com as competências exigidas na vaga.
- **Fator idioma:** 
Caso o candidato tenha os idiomas necessários da vaga então pontua, se não tiver, não pontua.


### **Formulário de vaga**
##### *Todas obrigatórias*
- id: gerado de forma sequencial a partir das vagas em aberto no folder
- data_abertura: a partir da data de criação da vaga
- data_fechamento: **input do consultor** (date)
- data_contratacao: **input do consultor** (data)
- prioridade_vaga: com base na data de fechamento - data de abertura
- consultor_responsavel: **input do consultor** (caixa selecao)
- empresa_contratante: **input do consultor** (nome)
- informacao_nome_solicitante: **input do consultor** (nome)
- contato_solicitante: **input do consultor** (numero em formato telefone Brasil (##) #####-####)
- titulo_vaga: **input do consultor** (descrito)
- nivel_profissional: **input do consultor** (nivel)
- area_atuacao: **input do consultor** (caixa de selecao)
- principais_atividades: **input do consultor** (caixa de selecao)
- tipo_contratacao: **input do consultor** (clt ou pj)
- prazo_contratacao: **input do consultor** (em meses, ou indeterminado)
- vaga_específica_pcd: **input do consultor** (binario)
- objetivo_vaga: **input do consultor** (descrição)
- pais_vaga: **input do consultor** (nome oficial)
- estado_vaga: **input do consultor** (nome oficial)
- cidade_vaga: **input do consultor** (nome oficial)
- cep_vaga: **input do consultor** (numero)
- modelo_trabalho: **input do consultor** (hibrido, presencial, remoto)
- disponibilidade_viagens: **input do consultor** (binario, se não for necessário, não aparecer no formulário candidato)
- nivel_academico_min: **input do consultor** (nivel)
- nivel_inglês_min: **input do consultor** (nivel, se não for necessário, não aparecer no formulário candidato)
- nivel_espanhol_min: **input do consultor** (nivel, se não for necessário, não aparecer no formulário candidato)
- hab_comportamentais: **input do consultor** (caixa de selecao)
- hab_tecnicas: **input do consultor** (caixa de selecao)
- beneficios:: **input do consultor** (colocar caixa de seleção com todos os benefícios que a empresa oferece)
- orcamento_salario: **input do consultor** (selecionar limites inferior e superior de salario total no mes)
- config_exibicao: **input do consultor** (quais informações acima deseja exibir no formulario do candidato, exemplo, se quer que apareça os beneficios, salários, descrição, competências, etc.)

### **Formulário de candidato**
- titulo_vaga: **input do candidato** (terá a lista de id-titulo_vaga no json de vagas, ao preencher aparecerá o formulário automático)
- nome: **input do candidato** 
- email: **input do candidato**
- contato: **input do candidato**
- pais: **input do candidato**
- estado: **input do candidato**
- cidade: **input do candidato**
- cep: **input do candidato**
- possui_def: **input do candidato**
- modelo_trabalho: **input do candidato**
- tipo_de_contrato: **input do candidato**
- disponibilidade_viagens: **input do candidato**
- nivel_academico: **input do candidato**
- area_atuacao: **input do candidato**
- tempo: **input do candidato**
- atividades: **input do candidato**
- nivel_ingles: **input do candidato**
- nivel_espanhol: **input do candidato**
- hab_comportamentais: **input do candidato**
- hab_tecnicas: **input do candidato**
- ultimo_salario: **input do candidato**
- ultimo_beneficio: **input do candidato**
- pretencao_salarial: **input do candidato**
- disponibidade_inicio: **input do candidato**
- anexo_curriculo: **input do candidato**
- id_vaga: gerado através do preenchimento do canditado no titulo_vaga
- codigo_candidato: valor unico vinculado ao email do candidato, sequencial unico pra todos os candidatos de todas as vagas, se o candidato inputar o mesmo email, buscar codigo já existente


#### Desafios
- Conseguir armezar formularios de vagas e linkar o formulario dos canditados a ele
- Treinar modelo
- Criar dashboard com a classificação dos candidatos por vaga




In [2]:
import pandas as pd
import shared.table_utils as tutils
pd.set_option('display.max_columns', None)

In [3]:
def load_json(df_read, colunms_transform: list):
    df = pd.read_json(df_read).T.reset_index()

    if df_read[6:] ==  "applicants.json":
        df = df.rename(columns={"index":"codigo_candidato"})
        df['codigo_candidato'] = df['codigo_candidato'].astype(int)
    else:
        df = df.rename(columns={"index":"id"})
        df['id'] = df['id'].astype(int)

    df_final = df.copy()
    for col in colunms_transform:
        
        df_norm = pd.json_normalize(df[col])
        df_norm.columns = [f"{col}_{subcol}" for subcol in df_norm.columns]
        df_final = pd.concat([df_final.drop(columns=[col]), df_norm], axis=1)
    
    df_final = df_final.loc[:, ~df_final.columns.duplicated()]
    
    if df_read[6:] ==  "prospects.json":
        result_df = df_final[['id']].copy()
        prospects_columns = [col for col in df_final.columns if col.startswith('prospects_')]
        result_df['prospects'] = df_final[prospects_columns].apply(
            lambda row: [item for item in row if item is not None and pd.notna(item)], 
            axis=1
        )
        result_df = result_df.explode('prospects').reset_index(drop=True)
        prospects_normalized = pd.json_normalize(result_df['prospects'])
        df_final = pd.concat([
            result_df[['id']], 
            prospects_normalized
        ], axis=1)
        
        df_final = df_final.dropna(subset=['nome'], how='all')
        df_final = df_final.rename(columns={"codigo":"codigo_candidato"})
        df_final['codigo_candidato'] = df_final['codigo_candidato'].astype(int)
    
    return df_final


In [4]:
applicants_df = load_json("Bases/applicants.json", ["infos_basicas", "informacoes_pessoais", "informacoes_profissionais", "formacao_e_idiomas"])
vagas_df = load_json("Bases/vagas.json", ["informacoes_basicas", "perfil_vaga", "beneficios"])
prospects_df = load_json("Bases/prospects.json", ["titulo", "modalidade", "prospects"])


In [5]:
vagas_df.head(5)

,id,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,18-05-2021,17-01-2022,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [6]:
applicants_df.head(5)

,codigo_candidato,cargo_atual,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso
0,31000,{},assistente administrativo\n\n\nsantosbatista\n...,,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,31000,Carolina Aparecida,Cadastro anterior ao registro de aceite,Carolina Aparecida,,:,carolina_aparecida@gmail.com,,0000-00-00,(11) 97048-2708,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN
1,31001,{},formação acadêmica\nensino médio (2º grau) em ...,,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,31001,Eduardo Rios,Cadastro anterior ao registro de aceite,Eduardo Rios,,Outros: Contato do RH,eduardo_rios@hotmail.com,,28-12-1994,(11) 93723-4396,,Feminino,Solteiro,Não,são paulo,,,,NaN,Analista Administrativo,Administrativa,,,,1900,,NaN,NaN,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN
2,31002,{},objetivo: área administrativa | financeira\n\n...,,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,31002,Pedro Henrique Carvalho,Cadastro anterior ao registro de aceite,Pedro Henrique Carvalho,,Anúncio:,pedro_henrique_carvalho@gmail.com,,12-12-1988,(11) 92399-9824,,Feminino,Solteiro,Não,são paulo,,,,NaN,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,NaN,NaN,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,,Administração de Empresas,2012,NaN
3,31003,{},formação\nensino médio completo\ninformática i...,,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,31003,Thiago Barbosa,Cadastro anterior ao registro de aceite,Thiago Barbosa,,Site de Empregos: Infojobs,thiago_barbosa@hotmail.com,,08-05-1992,(11) 98100-1727,,Feminino,Casado,Não,são paulo,,,,NaN,Área administrativa,Administrativa,,,,"1100,00",,NaN,NaN,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN
4,31004,{},última atualização em 09/11/2021\n­ sp\n\nensi...,,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,31004,Diogo das Neves,Cadastro anterior ao registro de aceite,Diogo das Neves,,:,diogo_das_neves@hotmail.com,,31-12-1969,(11) 92517-2678,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN


In [10]:
final_df = vagas_df.merge(prospects_df, on="id", how="left").merge(applicants_df, on="codigo_candidato", how="left")

import re # Vamos precisar da biblioteca de expressões regulares

# --- INÍCIO DO CÓDIGO DE LIMPEZA ---

# 1. Função para remover caracteres ilegais
def limpar_caracteres_ilegais(texto):
    """
    Esta função recebe um texto e remove os caracteres de controle que são
    inválidos em arquivos XML (formato base do .xlsx).
    Retorna o texto limpo ou o valor original se não for um texto.
    """
    # Verifica se o valor é uma string antes de tentar limpar
    if isinstance(texto, str):
        # Expressão regular para encontrar caracteres ilegais
        # Isso inclui a maioria dos caracteres de controle ASCII de 0 a 31,
        # exceto tabulação (9), nova linha (10) e retorno de carro (13) que são permitidos.
        caracteres_ilegais_re = re.compile(r'[\x00-\x08\x0b\x0c\x0e-\x1f]')
        return caracteres_ilegais_re.sub('', texto)
    else:
        # Se não for texto (ex: número, data, etc.), retorna o valor como está
        return texto

# 2. Aplicar a limpeza em todas as colunas de texto do DataFrame
# O ideal é aplicar apenas nas colunas que contêm texto (tipo 'object')
for coluna in final_df.select_dtypes(include=['object']).columns:
    print(f"Limpando a coluna: {coluna}")
    final_df[coluna] = final_df[coluna].apply(limpar_caracteres_ilegais)

print("\nLimpeza concluída com sucesso!")

# --- FIM DO CÓDIGO DE LIMPEZA ---


# 3. Agora, o seu comando para salvar o arquivo deve funcionar
print("Tentando salvar o arquivo Excel...")
final_df.to_excel("Bases/final_df.xlsx", index=False)
print("Arquivo 'final_df.xlsx' salvo com sucesso!")

Limpando a coluna: informacoes_basicas_data_requicisao
Limpando a coluna: informacoes_basicas_limite_esperado_para_contratacao
Limpando a coluna: informacoes_basicas_titulo_vaga
Limpando a coluna: informacoes_basicas_vaga_sap
Limpando a coluna: informacoes_basicas_cliente
Limpando a coluna: informacoes_basicas_solicitante_cliente
Limpando a coluna: informacoes_basicas_empresa_divisao
Limpando a coluna: informacoes_basicas_requisitante
Limpando a coluna: informacoes_basicas_analista_responsavel
Limpando a coluna: informacoes_basicas_tipo_contratacao
Limpando a coluna: informacoes_basicas_prazo_contratacao
Limpando a coluna: informacoes_basicas_objetivo_vaga
Limpando a coluna: informacoes_basicas_prioridade_vaga
Limpando a coluna: informacoes_basicas_origem_vaga
Limpando a coluna: informacoes_basicas_superior_imediato
Limpando a coluna: informacoes_basicas_nome
Limpando a coluna: informacoes_basicas_telefone
Limpando a coluna: informacoes_basicas_data_inicial
Limpando a coluna: informaco

In [9]:
final_df

,id,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,nome,codigo_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,cargo_atual,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,,Dante Nascimento,11010.0,Encaminhado ao Requisitante,11-05-2021,11-05-2021,,Srta. Bella Ferreira,{},\nsap basis s /4 hana - fiori especialista\n\n...,,,(19) 94137-8193,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,22-06-2018 15:01:14,pelo próprio candidato,dante_nascimento@hotmail.com,,Outros,08-12-2021 10:16:22,11010,Dante Nascimento,Cadastro anterior ao registro de aceite,Dante Nascimento,,Outros: Importação,dante_nascimento@hotmail.com,,31-12-1969,(19) 94137-8193,,Masculino,,,,,,,NaN,CONSULTOR BAS

In [7]:
final_df_vagas_contratados = final_df[final_df['situacao_candidado'].isin(["Contratado pela Decision",
                                                                            "Contratado como Hunting",
                                                                            "Aprovado",
                                                                            "Proposta Aceita"])]

list_vagas_contratados = list(final_df_vagas_contratados['id'].drop_duplicates())

list_vagas_contratados

final_df_all_analisar = final_df[final_df['id'].isin(list_vagas_contratados)]

num_candidato = final_df_all_analisar['id'].value_counts()
num_candidato = list(num_candidato[num_candidato >= 10].index)

final_df_amostrar = final_df_all_analisar[final_df_all_analisar['id'].isin(num_candidato)].sort_values(by='id').reset_index(drop=True)


final_df_amostrar

,id,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,nome,codigo_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,cargo_atual,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso
0,13,06-12-2018,00-00-0000,Suporte E-Transfer,Não,Gonzalez and Sons,Valentim Duarte,Decision São Paulo,Vitória Melo,Maria Helena Novaes,Cooperado,,Contratação,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,Cenesp,Sul,1000,Não,De: Até:,6x1 / madrugada 5x2,Pleno,Ensino Médio Incompleto,Nenhum,Nenhum,,Comercial-,Atendente help desk,nenhuma,Caso tenha PCD (com dificuldade apenas motora)...,,NaN,na,"1979,00 - p/ mês (168h)",mês,,Dr. Calebe Fogaça,12733.0,Desistiu,21-12-2018,26-12-2018,Não compareceu a entrevista agendada em 26/12/...,Liz Freitas,{},"26 anos, solteiro - são paulo, sp - zona oeste...",,,(11) 96042-7760,,21-12-2018 16:31:45,Srta. Bella Ferreira,dr._calebe_fogaça@gmail.com,,,21-12-2018 16:31:45,12733,Dr. Calebe Fogaça,Cadastro anterior ao registro de aceite,Dr. Calebe Fogaça,,:,dr._calebe_fogaça@gmail.com,,0000-00-00,(11) 96042-7760,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN
1,13,06-12-2018,00-00-0000,Suporte E-Transfer,Não,Gonzalez and Sons,Valentim Duarte,De

In [8]:
from shared.utils import tokenizer



In [1]:
import pandas as pd



In [ ]:
import pandas as pd
import glob
import os
import json

# Pastas
CANDIDATOS_PATH = "candidatos/"
VAGAS_PATH = "vagas/"

# --- 1. Ler todos os arquivos JSON ---
def ler_jsons(pasta):
    arquivos = glob.glob(os.path.join(pasta, "*.json"))
    dados = []
    for arq in arquivos:
        with open(arq, "r", encoding="utf-8") as f:
            conteudo = json.load(f)
            # Se o arquivo já for lista de objetos
            if isinstance(conteudo, list):
                dados.extend(conteudo)
            # Se for só um dicionário
            elif isinstance(conteudo, dict):
                dados.append(conteudo)
    return pd.DataFrame(dados)

df_candidatos = ler_jsons(VAGAS_PATH)
df_vagas = ler_jsons(VAGAS_PATH)

# --- 2. Merge LEFT por 'codigo_vaga' ---
df_final = pd.merge(
    df_candidatos,
    df_vagas,
    left_on="id_vaga",
    right_on="id",
    how="left",
    suffixes=("_candidato", "_vaga")
)

df_final




,id_vaga,nome,email_candidato,contato,pais,estado,cidade,possui_def,modelo_trabalho_candidato,tipo_contrato,...,disponibilidade_viagens_vaga,nivel_academico_min,nivel_ingles_min,nivel_espanhol_min,hab_comportamentais_vaga,hab_tecnicas_vaga,beneficios,orcamento_salario,status,pesos
0,1faced38,Lidiane,lidiane@gmail.com,7598989898,Brasil,BA,Feira de Santana,False,Presencial,CLT,...,False,Graduação,Básico,Não necessário,"[trabalho, equipe, lideranca, comunicacao]","[fechamento, folha, recrutamento, selecao, tre...","[vale, transporte, vale, alimentacao, plr, gym...","{'min': 4000, 'max': 6000}",encerrada,"{'tecnico': 0.2631578947368421, 'cultural': 0...."
1,a33ef51d,Luis Henrique,luis@gmail.com,75987878787,Brasil,BA,Feira de Santana,False,Presencial,CLT,...,False,Graduação,Fluente,Não necessário,"[lideranca, comunicacao, dono]","[analise, dados, kaizen, melhoria, processo, e...","[vale, alimentacao, vale, transporte, gym, pas...","{'min': 5000, 'max': 6000}",ativa,"{'tecnico': 0.2777777777777778, 'cultural': 0...."


In [ ]:
import pandas as pd
import glob
import os
import json
import streamlit as st

# --- 1. Ler todos os arquivos JSON ---
def ler_jsons(pasta):
    arquivos = glob.glob(os.path.join(pasta, "*.json"))
    dados = []
    for arq in arquivos:
        with open(arq, "r", encoding="utf-8") as f:
            conteudo = json.load(f)
            # Se o arquivo já for lista de objetos
            if isinstance(conteudo, list):
                dados.extend(conteudo)
            # Se for só um dicionário
            elif isinstance(conteudo, dict):
                dados.append(conteudo)
    return pd.DataFrame(dados)


def mostrar_resultados():
    st.title("📊 Resultados e Matchmaking")
    
    vagas = ler_jsons(VAGAS_PATH).to_dict(orient="records")
    candidatos = ler_jsons(CANDIDATOS_PATH).to_dict(orient="records")
    
    vagas_ordenadas = sorted(
        vagas, 
        key=lambda x: 0 if x.get('status') == 'ativa' else 1
    )

    vaga_selecionada = st.selectbox(
        "Selecione a Vaga para Análise", 
        options=vagas_ordenadas, 
        format_func=lambda x: f"#{x['id']} - {x['titulo_vaga']} - {'❌ Encerrada' if x.get('status') != 'ativa' else '✅ Ativa'}"
    )

    if vaga_selecionada:
        candidatos_vaga = [c for c in candidatos if c.get('id_vaga') == vaga_selecionada['id']]
        
        st.subheader(f"Candidatos para Vaga #{vaga_selecionada['id']}")
        st.write(f"**Total de candidatos:** {len(candidatos_vaga)}")
        
        if candidatos_vaga:
            candidatos_vaga.sort(key=lambda x: x.get('score_match', 0), reverse=True)
            
            for i, candidato in enumerate(candidatos_vaga, 1):
                status_atual = candidato.get('status_atual', 'Pendente')
                status_icon = '🟡' if status_atual == 'Pendente' else ('✅' if status_atual == 'Qualificado' else '❌')
                
                expander_label = (
                    f"#{i} - {candidato['nome']} - Score: {candidato.get('score_match', 0):.2%} - Status: **{status_icon} {status_atual}**"
                )
                
                with st.expander(expander_label):
                    col1, col2 = st.columns(2)
                    with col1:
                        st.write(f"**Email:** {candidato['email']}")
                        st.write(f"**Contato:** {candidato['contato']}")
                        st.write(f"**Área:** {candidato['areas_atuacao']}")
                        st.write(f"**Experiência:** {candidato['tempo_experiencia']} anos")
                    with col2:
                        st.write(f"**Inglês:** {candidato['nivel_ingles']}")
                        st.write(f"**Pretensão:** R$ {candidato['pretencao_salarial']:,.2f}")
                        st.write(f"**Modelo Trabalho:** {candidato['modelo_trabalho']}")
                        st.write(f"**Fatores de avaliação:** {candidato['fatores']}")
                    
                    # Histórico (mantém igual)
                    if 'historico_status' in candidato:
                        st.markdown("**Histórico de Avaliações:**")
                        historico_vaga = [h for h in candidato['historico_status'] if h.get('vaga_id') == vaga_selecionada['id']]
                        for hist in historico_vaga:
                            from datetime import datetime
                            data = datetime.fromisoformat(hist['data']).strftime('%d/%m/%Y %H:%M') 
                            st.info(f"[{data}] Status: **{hist['status']}** | Comentário: *{hist['comentario']}*")

    # --- NOVO BLOCO DE EXPORTAÇÃO GERAL ---
    st.markdown("---")
    st.subheader("📥 Exportar Resultados Gerais")

    # Converte listas para DataFrames
    df_candidatos = pd.DataFrame(candidatos)
    df_vagas = pd.DataFrame(vagas)

    # Merge
    df_final = pd.merge(
        df_candidatos,
        df_vagas,
        left_on="id_vaga",
        right_on="id",
        how="left",
        suffixes=("_candidato", "_vaga")
    )

    if st.button("📤 Exportar todos os dados para Excel"):
        filename = "dados_matchmaking.xlsx"
        df_final.to_excel(filename, index=False)
        st.success(f"✅ Arquivo {filename} exportado!")


if __name__ == "__main__":
    mostrar_resultados()


In [3]:
import os
import glob
import re
import pandas as pd
import docx  # Biblioteca python-docx
from pypdf import PdfReader # Biblioteca pypdf

def processar_curriculos(pasta_curriculos):
    """
    Lê arquivos .pdf e .docx de uma pasta, extrai o texto e o código do candidato.

    Args:
        pasta_curriculos (str): O caminho para a pasta contendo os arquivos de currículo.

    Returns:
        pd.DataFrame: Um DataFrame com as colunas 'codigo_candidato' e 'cv_pt'.
    """
    dados_curriculos = []
    
    # Pega todos os arquivos .pdf e .docx da pasta
    arquivos_pdf = glob.glob(os.path.join(pasta_curriculos, "*.pdf"))
    arquivos_docx = glob.glob(os.path.join(pasta_curriculos, "*.docx"))
    todos_arquivos = arquivos_pdf + arquivos_docx

    print(f"Encontrados {len(todos_arquivos)} currículos para processar.")

    for arquivo_path in todos_arquivos:
        try:
            # Extrai o nome do arquivo do caminho completo
            nome_arquivo = os.path.basename(arquivo_path)
            
            # --- Extração do código do candidato com Expressão Regular ---
            # O padrão (CAND\d+) busca pela palavra "CAND" seguida por um ou mais dígitos
            match = re.search(r"(CAND\d+)", nome_arquivo)
            if not match:
                print(f"AVISO: Não foi possível extrair o código do candidato do arquivo: {nome_arquivo}")
                continue # Pula para o próximo arquivo
            
            codigo_candidato = match.group(1)
            
            # --- Leitura do conteúdo do arquivo ---
            texto_completo = ""
            if nome_arquivo.endswith(".pdf"):
                reader = PdfReader(arquivo_path)
                for page in reader.pages:
                    texto_completo += page.extract_text() or "" # Adiciona o texto da página
            
            elif nome_arquivo.endswith(".docx"):
                documento = docx.Document(arquivo_path)
                for paragrafo in documento.paragraphs:
                    texto_completo += paragrafo.text + "\n" # Adiciona o texto do parágrafo
            
            # Armazena o resultado
            dados_curriculos.append({
                "codigo_candidato": codigo_candidato,
                "cv_pt": texto_completo.strip() # .strip() para remover espaços extras no início/fim
            })

        except Exception as e:
            print(f"ERRO ao processar o arquivo {nome_arquivo}: {e}")

    return pd.DataFrame(dados_curriculos)


# --- COMO UTILIZAR O CÓDIGO ---

# 1. Defina o caminho para a sua pasta de currículos
CURRICULOS_PATH = "curriculos/" # <--- SUBSTITUA AQUI

# Supondo que você já tenha seu DataFrame principal carregado
# Vou criar um exemplo para ilustrar
dados_principais = {
    'codigo_candidato': ['CAND1707', 'CAND1708', 'CAND1709'],
    'nome': ['Ana Silva', 'Bruno Costa', 'Carla Dias']
}
df_unificado = pd.DataFrame(dados_principais)


# 2. Chame a função para processar os currículos e criar o DataFrame de CVs
df_curriculos = processar_curriculos(CURRICULOS_PATH)

# 3. Junte o DataFrame de currículos ao seu DataFrame principal
if not df_curriculos.empty:
    df_final = pd.merge(
        df_unificado, 
        df_curriculos, 
        on="codigo_candidato", 
        how="left" # 'left' para manter todos os candidatos do df_unificado original
    )
else:
    print("Nenhum currículo foi processado, o DataFrame final não será modificado.")
    df_final = df_unificado
    df_final['cv_pt'] = None # Adiciona a coluna vazia para consistência


# 4. Verifique o resultado
print("\nDataFrame final com a coluna 'cv_pt':")
print(df_final.head())

# Para ver o conteúdo de um currículo específico:
# print(df_final.loc[df_final['codigo_candidato'] == 'CAND1707', 'cv_pt'].values[0])

Encontrados 1 currículos para processar.

DataFrame final com a coluna 'cv_pt':
  codigo_candidato         nome  \
0         CAND1707    Ana Silva   
1         CAND1708  Bruno Costa   
2         CAND1709   Carla Dias   

                                               cv_pt  
0  UNIVERSIDADE ESTADUAL DE FEIRA DE SANTANA – UE...  
1                                                NaN  
2                                                NaN  


In [6]:
import sys
print("Jupyter Python:", sys.executable)
print("Jupyter Path:", sys.path[:3])


Jupyter Python: c:\Users\dhlui\OneDrive\Área de Trabalho\Luis Henrique\ESTUDOS\FIAP\Tech Challenge\Fase 5\SeleIA\myenv\Scripts\python.exe
Jupyter Path: ['C:\\Users\\dhlui\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip', 'C:\\Users\\dhlui\\AppData\\Local\\Programs\\Python\\Python39\\DLLs', 'C:\\Users\\dhlui\\AppData\\Local\\Programs\\Python\\Python39\\lib']
